In [1]:
import psycopg2
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

from tqdm.notebook import tqdm_notebook as tqdm
from collections import OrderedDict

import json
import statistics

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_absolute_error

from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 12, 4

import pickle

%matplotlib inline

tqdm.pandas()

pd.options.display.max_columns = None

/Users/tim/miniconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
with open('config.json') as file:
    config = json.load(file)
    host = config['host']
    database = config['database']
    user = config['user']
    password = config['password']
engine = create_engine(f'postgresql://{user}:{password}@{host}:5432/{database}')

In [3]:
over_mae = pickle.load(open('over_mae.pickle', 'rb'))

In [13]:
array_string = "[" + ",".join(over_mae) + "]"

In [20]:
query = f"""
select * from ms_diagnosis md
where array [md."AdmtID"] <@ array {array_string}
"""

diagnoses_inaccurate = pd.read_sql_query(query, engine)

In [18]:
query = f"""
select * from ms_diagnosis md
where not array [md."AdmtID"] <@ array {array_string}
"""

diagnoses_accurate = pd.read_sql_query(query, engine)

In [35]:
set_inaccurate = set(diagnoses_inaccurate["DxDesc"].value_counts()[:25].index)

In [36]:
set_accurate = set(diagnoses_accurate["DxDesc"].value_counts()[:25].index)

In [42]:
inaccurate_uniques = set_inaccurate - set_accurate

In [43]:
inaccurate_uniques

{'Acidosis',
 'Acute kidney failure, unspecified',
 'Acute respiratory failure with hypoxia',
 'Cataract extraction status, unspecified eye',
 'Chronic kidney disease, stage 3 (moderate)',
 'Disruption of external operation (surgical) wound, NEC, init',
 'Do not resuscitate',
 'Encephalopathy, unspecified',
 'Exposure to other specified factors, initial encounter',
 'Personal history of other venous thrombosis and embolism',
 'Traum subdr hem w LOC of unsp duration, init',
 'Type 2 diabetes mellitus w diabetic chronic kidney disease',
 'Type 2 diabetes mellitus with hyperglycemia',
 'Type 2 diabetes w diabetic peripheral angiopathy w gangrene'}